In [ ]:
import os

import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from pyia import GaiaData

In [ ]:
hip = at.Table.read('/Users/apricewhelan/data/Hipparcos/hip2.fits')

In [ ]:
xhip = at.Table.read('/Users/apricewhelan/data/Hipparcos/xhip.fits')
xhip.meta = None

In [ ]:
pulkovo = at.Table.read('/Users/apricewhelan/data/Hipparcos/Pulkovo.fits')
pulkovo.meta = None
pulkovo.remove_columns(['eRV', '_RA', '_DE', 'recno'])
pulkovo['q_RV'] = 3

In [ ]:
all_rv = at.vstack((xhip, pulkovo[~np.isin(pulkovo['HIP'], xhip['HIP'])]))
hipall = at.join(hip, all_rv, keys='HIP', 
                 uniq_col_name='{col_name}{table_name}',
                 table_names=['', '_2'],
                 join_type='left')

In [ ]:
len(hip), len(hipall)

In [ ]:
(xhip['q_RV'] < 3).sum()

In [ ]:
hip_like_gaia = at.Table()

hip_like_gaia['source_id'] = hipall['HIP']
hip_like_gaia['ra'] = hipall['RArad']
hip_like_gaia['dec'] = hipall['DErad']
hip_like_gaia['parallax'] = hipall['Plx']
hip_like_gaia['pmra'] = hipall['pmRA']
hip_like_gaia['pmdec'] = hipall['pmDE']

hip_like_gaia['radial_velocity'] = hipall['RV']
bad_rv = (hipall['q_RV'] >= 3) | (hipall['e_RV'] < 0.1)
hip_like_gaia['radial_velocity'][bad_rv] = np.nan

hip_like_gaia['ra_error'] = hipall['e_RArad']
hip_like_gaia['dec_error'] = hipall['e_DErad']
hip_like_gaia['parallax_error'] = hipall['e_Plx']
hip_like_gaia['pmra_error'] = hipall['e_pmRA']
hip_like_gaia['pmdec_error'] = hipall['e_pmDE']
hip_like_gaia['radial_velocity_error'] = hipall['e_RV']
hip_like_gaia['radial_velocity_error'][bad_rv] = np.inf

for i, name1 in enumerate(['ra', 'dec', 'parallax', 'pmra', 'pmdec']):
    for j, name2 in enumerate(['ra', 'dec', 'parallax', 'pmra', 'pmdec']):
        if i >= j: continue
        hip_like_gaia[f"{name1}_{name2}_corr"] = 0.

In [ ]:
hip_like_gaia.write('../data/hip_like_gaia.fits', overwrite=True)